In [4]:
from langchain.document_loaders import CSVLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

In [5]:
embedding_function=HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

C:\Users\Administrator\AppData\Local\Temp\ipykernel_16976\281083725.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_function=HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
d:\MuhammetAliVarlik\Anaconda\envs\MovieScout\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [6]:
loader = CSVLoader("../data/processed/TMDB.csv",encoding="utf-8")
documents = loader.load()

In [7]:
import os

persist_directory = "../models/chroma_db"

# Check if the directory for the Chroma database exists
if os.path.exists(persist_directory):
    # Load the existing Chroma database
    db = Chroma(persist_directory=persist_directory, embedding_function=embedding_function)
    print("Existing Chroma database loaded.")
else:
    # If the database doesn't exist, create a new one
    db = Chroma.from_documents(documents, embedding_function, persist_directory=persist_directory)
    # Persist the new database
    db.persist()
    print("New Chroma database created and persisted.")


C:\Users\Administrator\AppData\Local\Temp\ipykernel_16976\224117220.py:8: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  db = Chroma(persist_directory=persist_directory, embedding_function=embedding_function)


Existing Chroma database loaded.


In [8]:
query = "I want to watch a sci-fi movie."
docs = db.similarity_search(query)
print(docs[0].page_content)

index: 2141
title: Evolution
vote_average: 6.051
vote_count: 2022
status: Released
release_date: 2001-06-08
runtime: 101
overview: A comedy that follows the chaos that ensues when a meteor hits the Earth carrying alien life forms that give new meaning to the term "survival of the fittest." David Duchovny, Orlando Jones, Seann William Scott, and Julianne Moore are the only people standing between the aliens and world domination... which could be bad news for the Earth.
poster_path: /dwIrP544LxJtXToLpzkEa08pIMm.jpg
genres: Comedy, Science Fiction, Action
backdrop_path: /uQVXQ7VI0jbLDo5QwGGyLQUd1so.jpg


In [9]:
from langchain.prompts import ChatPromptTemplate,PromptTemplate
from langchain_community.chat_models import ChatLlamaCpp
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks import StreamingStdOutCallbackHandler

In [31]:
"""
Phi-3.5-mini-instruct-Q5_K_L.gguf
tinyllama-1.1b-chat-v1.0.Q4_K_M.gguf
Phi-3-mini-4k-instruct-q4.gguf
Llama-3.2-1B-Instruct-Q6_K_L.gguf
"""

# Initialize Callback manager
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])
llm = ChatLlamaCpp(
    model_path="../models/Llama-3.2-1B-Instruct-Q6_K_L.gguf",
    use_mlock=True,
    callback_manager=callback_manager,
    n_ctx=1024,  # Daha küçük bağlam
    verbose=True,
    n_gpu_layers=48,
    n_threads=16,
    n_batch=1024  # Daha küçük batch size
)


llama_model_loader: loaded meta data with 35 key-value pairs and 147 tensors from ../models/Llama-3.2-1B-Instruct-Q6_K_L.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Llama 3.2 1B Instruct
llama_model_loader: - kv   3:                           general.finetune str              = Instruct
llama_model_loader: - kv   4:                           general.basename str              = Llama-3.2
llama_model_loader: - kv   5:                         general.size_label str              = 1B
llama_model_loader: - kv   6:                            general.license str              = llama3.2
llama_model_loader: - kv   7:           

In [32]:
template = """
You are a movie recommendation system. Based on the context provided, recommend at least three movies that fit the user's needs.

Context: {context}
Question: {question}

Instructions: Respond in 1-2 sentences without spoilers. Reccomend only 1 movie. Format your answer as follows:

<p>- Your opinion about the question(1-2 sentence). </p>
<li>
<h5>Movie Name 1</h5> <h4>(Release Date) | Genres | Vote Average</h4>
<p>Brief overview of the movie.(1 sentence)</p>
</li>
"""

prompt = PromptTemplate(
            template=template, 
            input_variables=['context', 'question']
        )

In [33]:
from langchain.chains import RetrievalQA

In [34]:
retriever = db.as_retriever(search_kwargs={"k": 2})
chain = RetrievalQA.from_chain_type(
            llm,
            retriever=retriever,
            return_source_documents=False,
            chain_type_kwargs={"prompt": prompt}
        )

In [35]:
chain.invoke("Recommend a sci-fi movies like Star Wars")

Based on the context of a sci-fi movie similar to Star Wars, I recommend "The Matrix" (1999). It is an iconic science fiction film that explores themes of rebellion against a powerful computer system.

llama_perf_context_print:        load time =   13949.95 ms
llama_perf_context_print: prompt eval time =       0.00 ms /   524 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /    42 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =   17769.51 ms /   566 tokens


{'query': 'Recommend a sci-fi movies like Star Wars',
 'result': 'Based on the context of a sci-fi movie similar to Star Wars, I recommend "The Matrix" (1999). It is an iconic science fiction film that explores themes of rebellion against a powerful computer system.'}

In [36]:
response = chain.invoke("Recommend an commedy movie. But it should include rommence theme also")

Llama.generate: 60 prefix-match hit, remaining 402 prompt tokens to eval


I recommend "Zoey 102" as a comedy movie that incorporates romantic themes. This 2023 film is about a high school student navigating love, friendships, and her own identity.

llama_perf_context_print:        load time =   13949.95 ms
llama_perf_context_print: prompt eval time =       0.00 ms /   402 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /    39 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =   14669.98 ms /   441 tokens


In [37]:
response = chain.invoke("Recommend a movie with a theme of friendship")

Llama.generate: 60 prefix-match hit, remaining 402 prompt tokens to eval


- I think a great movie recommendation for this theme is "The Descendants" (2011), where the main characters form an unlikely friendship despite their different backgrounds.

llama_perf_context_print:        load time =   13949.95 ms
llama_perf_context_print: prompt eval time =       0.00 ms /   402 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /    33 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =   15680.40 ms /   435 tokens


In [38]:
response = chain.invoke("I want to watch a superhero movie with a female main star")

Llama.generate: 60 prefix-match hit, remaining 413 prompt tokens to eval


I recommend watching "Captain Marvel" as it stars Carol Danvers, a powerful and iconic superhero. This movie is also set in the 1990s, which might appeal to fans of the original Superman movie released in 1978.

llama_perf_context_print:        load time =   13949.95 ms
llama_perf_context_print: prompt eval time =       0.00 ms /   413 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /    48 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =   16036.69 ms /   461 tokens


In [39]:
response = chain.invoke("I want you to recommend movies that are guaranteed to make you cry.")

Llama.generate: 60 prefix-match hit, remaining 441 prompt tokens to eval


- I think you should recommend the movie "The Notebook" (2004) directed by Nick Cassavetes, which is a classic romantic drama that many people have cried over.

llama_perf_context_print:        load time =   13949.95 ms
llama_perf_context_print: prompt eval time =       0.00 ms /   441 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /    36 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =   16889.75 ms /   477 tokens
